# Adding resveratrol-producing pathways

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [5]:
model = read_sbml_model("../data/models/model_Y8_bigg_complaint.xml")

In [6]:
model

Name,yeastGEM_v8__46__6__46__2
Memory address,0x0118616c80
Number of metabolites,2744
Number of reactions,4063
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


## Model 1: Resveratrol from glucose 

In [11]:
model.metabolites.atp_c


Metabolite identifier,atp_c
Name,ATP
Memory address,0x1185fa8c0
Formula,C10H12N5O13P3
Compartment,c
In 182 reaction(s),"STCOATxc, NNATr, PYDAMK, METAT, UREASE, ATPtr, GTHRDtv, r_4624, CHOLK, PROTRS_1, PYDXNK, OPAH, FATACYCOATRAABC181, METTRS, r_4619, GLUTRS, PMEVK, PTPATi, CBPS, URIK1, ATPS, LYSTRS, LIPATPT, PMPK,..."


In [9]:
## Add reactions
# Reaction 1 (HA-TAL):L-tyrosine <=> (E).4.coumarate +   NH4+
new_reaction1 = Reaction('HA-TAL') # 3-(o)xopropanoate:(N)ADP+ (o)xidoreductase
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction1.add_metabolites({model.metabolites.tyr__L_c: -1, 
                               coumarate4: 1, # 3-Oxopropanoate
                               model.metabolites.nh4_c: 1, # Search fpr NH4 ID
                              })
model.add_reactions([new_reaction1])

In [10]:
model.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x11a3db340
Formula,None
Compartment,c
In 1 reaction(s),HA-TAL


In [12]:
new_reaction2 = Reaction('AT-4CL1')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2.add_metabolites({model.metabolites.coumarate4: -1,
                               model.metabolites.atp_c: -1,
                               model.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model.metabolites.amp_c: 1,
                               model.metabolites.ppi_c: 1,
                              })
model.add_reactions([new_reaction2])

In [13]:
new_reaction3 = Reaction('VVVST1')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3.add_metabolites({model.metabolites.e4_coumaroyl_coa: -1,
                               model.metabolites.h_c: -3,
                               model.metabolites.malcoa_c: -3,
                               model.metabolites.co2_c: 4,
                               model.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model.add_reactions([new_reaction3])

In [22]:
model.reactions.EX_glc__D_e.bounds

(-1.0, 1000.0)

In [14]:
model.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x11a3d8310
Stoichiometry,e4_coumaroyl_coa + 3 h_c + 3 malcoa_c --> 4 co2_c + 4 coa_c + trans_resv + 3 H+ + 3 malonyl-CoA --> 4 carbon dioxide + 4 coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [23]:
model.add_boundary(model.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x11a3db4f0
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [24]:
from cobra.io import write_sbml_model
write_sbml_model(model,'../data/models/yeast8_resv_glc.xml')

from cobra.io import write_sbml_model
write_sbml_model(model,'data/models/yeast_resv_glc.xml')

In [25]:
with model:
    model.objective=model.reactions.VVVST1
    lets_produce_resv= model.optimize().objective_value
lets_produce_resv

0.3103760811914647

In [26]:
model

Name,yeastGEM_v8__46__6__46__2
Memory address,0x0118616c80
Number of metabolites,2747
Number of reactions,4067
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


## Model 2: Resveratrol from coumaric acid from https://doi.org/10.1128/AEM.00609-06

In [69]:
model_resv_cou = read_sbml_model('../data/models/iMM904.xml')

In [70]:
new_reaction2_1 = Reaction('AT-4CL3')
coumarate4 = Metabolite(id='coumarate4', compartment='c')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2_1.add_metabolites({coumarate4: -1,
                               model_resv_cou.metabolites.atp_c: -1,
                               model_resv_cou.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_cou.metabolites.amp_c: 1,
                               model_resv_cou.metabolites.ppi_c: 1,
                              })

model_resv_cou.add_reactions([new_reaction2_1])

In [71]:
model_resv_cou.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x124ba7730
Formula,None
Compartment,c
In 1 reaction(s),AT-4CL3


In [72]:
new_reaction2_2 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction2_2.add_metabolites({model_resv_cou.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_cou.metabolites.h_c: -3,
                               model_resv_cou.metabolites.malcoa_c: -3,
                               model_resv_cou.metabolites.co2_c: 4,
                               model_resv_cou.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_cou.add_reactions([new_reaction2_2])

In [75]:
new_reaction2_3 = Reaction('Ex_coumarate_transport')
coumarate4_e = Metabolite(id='coumarate4_e', compartment='e')
new_reaction2_3.add_metabolites({coumarate4_e: -1,
                               model_resv_cou.metabolites.coumarate4: 1
                              })
model_resv_cou.add_reactions([new_reaction2_3])

Ignoring reaction 'Ex_coumarate_transport' since it already exists.


In [76]:
#how much resveratrol is produced?
model_resv_cou.add_boundary(model_resv_cou.metabolites.trans_resv, type='demand')

ValueError: Boundary reaction 'DM_trans_resv' already exists.

In [77]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_cou,'../data/models/yeast_resv_cou.xml')

In [78]:
model_resv_cou

Name,iMM904
Memory address,0x012381ef50
Number of metabolites,1230
Number of reactions,1581
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


## Model 3: Resveratrol from Phenylalanine (https://doi.org/10.1016/j.ymben.2009.07.004)


In [ ]:
model_resv_phen = read_sbml_model('../data/models/iMM904.xml')

In [ ]:
new_reaction3_1 = Reaction('PAL3')
e_cinnamate_c = Metabolite(id='e_cinnamate_c', compartment='c')
new_reaction3_1.add_metabolites({model_resv_phen.metabolites.phe__L_c: -1,
                               e_cinnamate_c: 1,
                               model_resv_phen.metabolites.nh4_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_1])

In [ ]:
new_reaction3_2 = Reaction('C4H') 
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction3_2.add_metabolites({model_resv_phen.metabolites.e_cinnamate_c: -1,
                               model_resv_phen.metabolites.fmnh2_c: -1,
                               model_resv_phen.metabolites.o2_c: -1,
                               coumarate4: 1, 
                               model_resv_phen.metabolites.h2o_c: 1,
                               model_resv_phen.metabolites.h_c: 1,
                               model_resv_phen.metabolites.fmn_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_2])

In [ ]:
new_reaction3_3 = Reaction('AT-4CL2')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction3_3.add_metabolites({model_resv_phen.metabolites.coumarate4: -1,
                               model_resv_phen.metabolites.atp_c: -1,
                               model_resv_phen.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_phen.metabolites.amp_c: 1,
                               model_resv_phen.metabolites.ppi_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_3])

In [ ]:
new_reaction3_4 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3_4.add_metabolites({model_resv_phen.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_phen.metabolites.h_c: -3,
                               model_resv_phen.metabolites.malcoa_c: -3,
                               model_resv_phen.metabolites.co2_c: 4,
                               model_resv_phen.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_phen.add_reactions([new_reaction3_4])

In [ ]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_phen,'../data/models/yeast_resv_phen.xml')

In [ ]:
model_resv_phen.add_boundary(model_resv_phen.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x13a1179a0
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


### How much resveratrol is porduced?

In [ ]:
# Model using glucose as subsrate
with model_resv_glc:
    model_resv_glc.objective=model_resv_glc.reactions.VVVST1
    lets_produce_resv= model_resv_glc.optimize().objective_value
lets_produce_resv

1.178571428571429

In [ ]:
# Model using phenylananine
with model_resv_phen:
    model_resv_phen.objective=model_resv_phen.reactions.STS
    lets_produce_resv= model_resv_phen.optimize().objective_value
lets_produce_resv

1.0749185667752432

In [48]:
# model usin coumaric acid as substrate
with model_resv_cou:
    model_resv_cou.objective=model_resv_cou.reactions.STS
    lets_produce_resv= model_resv_cou.optimize().objective_value
lets_produce_resv

-3.389454704016765e-33